In [11]:
import os
# Run export OPENAI_API_KEY=sk-YOUR_OPENAI_API_KEY...
# Get openAI api key by reading local .env file
from dotenv import load_dotenv, find_dotenv
dotenv_path = os.path.abspath('../../../.env')
_ = load_dotenv(dotenv_path)
OPENAI_API_KEY  = os.environ['OPENAI_API_KEY']
PG_DB_PW = os.environ['POSTGRES_DB_PASSWORD']
# host= os.environ['POSTGRES_DB_HOST']
host= 'localhost'
port= os.environ['POSTGRES_DB_PORT']
user= os.environ['POSTGRES_DB_USER']
password= os.environ['POSTGRES_DB_PASSWORD']
dbname= os.environ['POSTGRES_DB_DBNAME']


In [19]:
import openai
import os
import pandas as pd
import numpy as np
import json
import tiktoken
import psycopg2
import ast
import pgvector
import math
from psycopg2.extras import execute_values
from pgvector.psycopg2 import register_vector

from IPython.display import Markdown, display

import pandas as pd

from langchain.vectorstores.pgvector import PGVector
from langchain.sql_database import SQLDatabase
# from langchain_experimental.sql import SQLDatabaseChain
from langchain_openai import ChatOpenAI, OpenAI
from langchain.chains import RetrievalQA
from langchain_openai.embeddings import OpenAIEmbeddings


In [30]:
# CONNECTION_STRING = f"postgresql://{user}:{password}@{host}:{port}/{dbname}"
# CONNECTION_STRING = f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}?sslmode=require"
CONNECTION_STRING = f"postgresql://{user}:{password}@{host}:{port}/{dbname}"
collection_name= "arxiv"


In [33]:
import psycopg2

# Replace these with your values
host = "localhost"  # Use "localhost" if connecting from the host machine

# Create the connection
try:
    conn = psycopg2.connect(
        host=host,
        port=port,
        user=user,
        password=password,
        database=dbname,
        connect_timeout=3
    )
    print("Connection successful!")
except Exception as e:
    print(f"Error: {e}")
""

Connection successful!


''

In [14]:
# Connect to PostgreSQL database in Timescale using connection string
conn = psycopg2.connect(CONNECTION_STRING)
cur = conn.cursor()

#install pgvector
cur.execute("CREATE EXTENSION IF NOT EXISTS vector");
conn.commit()

In [15]:
embedding_df = pd.read_csv('../../data-ingest/data/embeddings/test_embedding.csv', index_col=0)

In [16]:
embedding_df

,title,content,tokens,embeddings
0,pdf3.pdf,Detection of stellar light from quasar host ga...,1037,"[-0.0059149968437850475, -0.006691891700029373..."
1,pdf3.pdf,"22Department of Astronomy, School of Science, ...",847,"[-0.022671174257993698, 0.023965613916516304, ..."
2,pdf3.pdf,"3.4× 1010 M⊙, respectively), compact, and disk...",634,"[-0.045199356973171234, 0.013202028349041939, ..."
3,pdf3.pdf,a point-like quasar and an extended host. The ...,603,"[-0.04193825647234917, 0.0002854476042557508, ..."
4,pdf3.pdf,"of young and relatively old stars, and thus to...",613,"[-0.01969219371676445, 0.014046339318156242, -..."
...,...,...,...,...
133,pdf1.pdf,"by known stars. In the remaining images above,...",925,"[0.002194813219830394, 0.0010228499304503202, ..."
134,pdf1.pdf,0.006 0.104 ± 0.012 0.142 ± 0.030 ... 0.006 ± ...,1023,"[-0.024804944172501564, 0.011800343170762062, ..."
135,pdf1.pdf,15.0 0.003 ± 0.002 0.009 ± 0.003 0.019 ± 0.003...,1061,"[-0.01906454563140869, -0.009697528555989265, ..."
136,pdf1.pdf,together. 24 Table 9.Applied confusion correct...,1107,"[-0.016655320301651955, -0.006832951679825783,..."


In [17]:
#load documents from Pandas dataframe for insertion into database
from langchain.document_loaders import DataFrameLoader

# page_content_column is the column name in the dataframe to create embeddings for
loader = DataFrameLoader(embedding_df, page_content_column = 'content')
docs = loader.load()


In [18]:
type(docs[0])

langchain_core.documents.base.Document

In [20]:
embeddings = OpenAIEmbeddings()

In [21]:
embedding_df['content']

0      Detection of stellar light from quasar host ga...
1      22Department of Astronomy, School of Science, ...
2      3.4× 1010 M⊙, respectively), compact, and disk...
3      a point-like quasar and an extended host. The ...
4      of young and relatively old stars, and thus to...
                             ...                        
133    by known stars. In the remaining images above,...
134    0.006 0.104 ± 0.012 0.142 ± 0.030 ... 0.006 ± ...
135    15.0 0.003 ± 0.002 0.009 ± 0.003 0.019 ± 0.003...
136    together. 24 Table 9.Applied confusion correct...
137    the structure function slope and PSD slope in ...
Name: content, Length: 138, dtype: object

In [22]:
from langchain.vectorstores.pgvector import DistanceStrategy

db = PGVector.from_documents(
    documents=docs,
    embedding=embeddings,
    collection_name=collection_name,
    connection_string=CONNECTION_STRING,
    distance_strategy=DistanceStrategy.COSINE,
    pre_delete_collection=False 
)

/opt/anaconda3/envs/astro-buddy/lib/python3.12/site-packages/langchain_community/vectorstores/pgvector.py:487: LangChainPendingDeprecationWarning: Please use JSONB instead of JSON for metadata. This change will allow for more efficient querying that involves filtering based on metadata. Please note that filtering operators have been changed when using JSONB metadata to be prefixed with a $ sign to avoid name collisions with columns. If you're using an existing database, you will need to create a db migration for your metadata column to be JSONB and update your queries to use the new operators. 
  store = cls(


In [23]:
PGVector.__dict__

mappingproxy({'__module__': 'langchain_community.vectorstores.pgvector',
              '__doc__': '.. deprecated:: 0.0.31 This class is pending deprecation and may be removed in a future version. You can swap to using the `PGVector` implementation in `langchain_postgres`. Please read the guidelines in the doc-string of this class to follow prior to migrating as there are some differences between the implementations. See <https://github.com/langchain-ai/langchain-postgres> for details about the new implementation. Use :class:`~from langchain_postgres import PGVector;` instead. \n\n`Postgres`/`PGVector` vector store.\n\n**DEPRECATED**: This class is pending deprecation and will likely receive\n    no updates. An improved version of this class is available in\n    `langchain_postgres` as `PGVector`. Please use that class instead.\n\n    When migrating please keep in mind that:\n        * The new implementation works with psycopg3, not with psycopg2\n          (This implementation does not

In [24]:
from langchain.schema import Document

# Query for which we want to find semantically similar documents
query = "What is a quasar?"

#Fetch the k=3 most similar documents
docs =  db.similarity_search(query, k=3)


In [25]:
docs

[Document(metadata={'title': 'pdf2.pdf', 'tokens': 548, 'embeddings': '[-0.0028234771452844143, 0.03425675630569458, 0.014410953968763351, 0.013279718346893787, 0.012812469154596329, 0.012468180619180202, -0.02525605820119381, 0.0051766312681138515, -0.038511183112859726, 0.003479778068140149, -0.0025360574945807457, 0.009965936653316021, -0.061726100742816925, -0.010691403411328793, 0.04876607656478882, 0.03103519417345524, -0.03725698962807655, 0.005822172854095697, -0.005121299065649509, 0.0330517441034317, 0.015333157032728195, -0.03912598639726639, -0.02299358695745468, 0.020386828109622, -0.09187597781419754, -0.030641721561551094, 0.02117377333343029, 0.036765147000551224, 0.006025057751685381, -0.01548070926219225, 0.018874414265155792, -0.026338109746575356, -0.04170815274119377, 0.006212572101503611, -0.05459439754486084, 0.06920208781957626, -0.026780765503644943, -0.0429377555847168, -0.035068295896053314, 0.0012603437062352896, 0.013169054873287678, -0.014054369181394577, 

In [26]:
# Create retriever from database
# We specify the number of results we want to retrieve (k=3)
retriever = db.as_retriever(
    search_kwargs={"k": 3}
    )


In [27]:
llm = ChatOpenAI(temperature = 0.0, model = 'gpt-3.5-turbo-16k')


In [28]:
from langchain.chains import RetrievalQA
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever,
    verbose=True,
)


In [29]:
query =  "What is a quasar?"

response = qa_stuff.run(query)

from IPython.display import Markdown, display
display(Markdown(response))


/var/folders/qx/bpj16cl90cq20swjd4j79cdh0000gn/T/ipykernel_40666/27716869.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_stuff.run(query)




> Entering new RetrievalQA chain...

> Finished chain.


A quasar, short for "quasi-stellar radio source," is a highly energetic and luminous object that is powered by the accretion of matter onto a supermassive black hole at the center of a galaxy. Quasars are the most luminous objects in the universe and emit vast amounts of energy across the electromagnetic spectrum, from radio waves to X-rays. They are typically found in the early universe and provide valuable insights into the formation and evolution of galaxies, as well as the intergalactic medium.

In [ ]:
# New chain to return context and sources
qa_stuff_with_sources = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever,
    return_source_documents=True,
    verbose=True,
)

query =  "What is the evidence for an enhanced accretion episode from Sgr A* in 2019?"

# To run the query, we use a different syntax since we're returning more than just the response text
responses = qa_stuff_with_sources({"query": query})


In [ ]:
source_documents = responses["source_documents"]
source_content = [doc.page_content for doc in source_documents]
source_metadata = [doc.metadata for doc in source_documents]

# Construct a single string with the LLM output and the source titles and urls
def construct_result_with_sources():
    result = responses['result']
    result += "\n\n"
    result += "Sources used:"
    for i in range(len(source_content)):    
        result += "\n\n"
        result += source_metadata[i]['title']
        result += "\n\n"
        return result

display(Markdown(construct_result_with_sources()))


In [ ]:
query =  "What distinguishes a quasar from a normal black hole?"

# To run the query, we use a different syntax since we're returning more than just the response text
responses = qa_stuff_with_sources({"query": query})

source_documents = responses["source_documents"]
source_content = [doc.page_content for doc in source_documents]
source_metadata = [doc.metadata for doc in source_documents]

# Construct a single string with the LLM output and the source titles and urls
def construct_result_with_sources():
    result = responses['result']
    result += "\n\n"
    result += "Sources used:"
    for i in range(len(source_content)):    
        result += "\n\n"
        result += source_metadata[i]['title']
        result += "\n\n"
        return result

display(Markdown(construct_result_with_sources()))


In [ ]:
# db = PGVector.from_documents(
#     embedding=embeddings,
#     documents=docs,
#     collection_name=collection_name,
#     connection_string=CONNECTION_STRING,
#     pre_delete_collection=False,
# )


In [6]:
from langchain_openai.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

store = PGVector(
    collection_name=collection_name,
    connection_string=CONNECTION_STRING,
    embedding_function=embeddings,
)

In [14]:
pdf_texts_df = pd.read_csv('../../data-ingest/data/text/pdf_texts.csv', index_col=0)

In [16]:
pdf_texts_df['content']

0    Detection of stellar light from quasar host ga...
1    Quasars and the\nIntergalactic Medium at\nCosm...
2    Draft version August 22, 2023\nTypeset using L...
Name: content, dtype: object

In [19]:
store.add_texts(pdf_texts_df['content'])

['06f9f18a-bfa6-11ee-8a3d-9801a78f9833',
 '06f9fdf6-bfa6-11ee-8a3d-9801a78f9833',
 '06f9fee6-bfa6-11ee-8a3d-9801a78f9833']

In [20]:
schema = {
    "properties": {
        "article_title": {"type": "string"},
        "authors": {"type": "string"},
        "publish_date": {"type": "string"},
    },
    "required": ["article_title", "authors", "publish_date"],
}

In [29]:
store.add_texts(pdf_texts_df['content'], metadatas=schema)

['113d7142-bfa8-11ee-8a3d-9801a78f9833',
 '113d72c8-bfa8-11ee-8a3d-9801a78f9833',
 '113d7340-bfa8-11ee-8a3d-9801a78f9833']

In [26]:
store.delete_collection()

In [ ]:
store

In [ ]:
store.add_documents(docs)

In [7]:
retriever = store.as_retriever()
llm = ChatOpenAI(temperature = 0.0, model = 'gpt-3.5-turbo-16k')


In [8]:
# New chain to return context and sources
qa_stuff_with_sources = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever,
    return_source_documents=True,
    verbose=True,
)

query =  "How does a quasar die?"

# To run the query, we use a different syntax since we're returning more than just the response text
responses = qa_stuff_with_sources({"query": query})


/Users/dangause/anaconda3/envs/langchain/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...

> Finished chain.


In [9]:
source_documents = responses["source_documents"]
source_content = [doc.page_content for doc in source_documents]
source_metadata = [doc.metadata for doc in source_documents]

# Construct a single string with the LLM output and the source titles and urls
def construct_result_with_sources():
    result = responses['result']
    result += "\n\n"
    result += "Sources used:"
    for i in range(len(source_content)):    
        result += "\n\n"
        result += source_metadata[i]['title']
        result += "\n\n"
        return result

display(Markdown(construct_result_with_sources()))

The exact process of how a quasar dies is not yet fully understood. Quasars are powered by the accretion of matter onto supermassive black holes at the centers of galaxies. This accretion process releases a tremendous amount of energy, causing the quasar to shine brightly. However, quasars are not expected to accrete matter at a constant rate throughout their entire lifetime. Instead, they are believed to exhibit a flickering light curve, with episodes of high and low/no accretion.

As a quasar transitions from an active phase to a quiescent phase, its accretion rate decreases, and it gradually fades away. The exact mechanisms that regulate the accretion process and determine the lifetime of a quasar are still the subject of ongoing research. It is possible that factors such as the availability of gas and the dynamics of the surrounding environment play a role in the quasar's demise.

Additionally, it is important to note that the term "quasar" is often used to refer to the active phase of a supermassive black hole, while the term "inactive" or "dormant" black hole is used to describe the quiescent phase. During the quiescent phase, the black hole may still exist, but it is not actively accreting matter and therefore does not emit significant amounts of radiation.

Sources used:

chronicling the reionization history at  6 lesssim z  lesssim 7  with   emergent quasar damping wings



In [10]:
construct_result_with_sources()

'The exact process of how a quasar dies is not yet fully understood. Quasars are powered by the accretion of matter onto supermassive black holes at the centers of galaxies. This accretion process releases a tremendous amount of energy, causing the quasar to shine brightly. However, quasars are not expected to accrete matter at a constant rate throughout their entire lifetime. Instead, they are believed to exhibit a flickering light curve, with episodes of high and low/no accretion.\n\nAs a quasar transitions from an active phase to a quiescent phase, its accretion rate decreases, and it gradually fades away. The exact mechanisms that regulate the accretion process and determine the lifetime of a quasar are still the subject of ongoing research. It is possible that factors such as the availability of gas and the dynamics of the surrounding environment play a role in the quasar\'s demise.\n\nAdditionally, it is important to note that the term "quasar" is often used to refer to the activ

ModuleNotFoundError: No module named 'src'

In [18]:
%load_ext autoreload
%autoreload 2
import os
import sys
module_path = os.path.abspath(os.path.join('../../src')) # or the path to your source code
sys.path.insert(0, module_path)
import src

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ModuleNotFoundError: No module named 'src'

In [19]:
nb_dir = os.path.split(os.getcwd())[0]


In [20]:
nb_dir

'/Users/dangause/Desktop/personal_projects/astro_gpt/astro-buddy/services/gpt-connection'

In [21]:
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [28]:
from src.config import settings

ValidationError: 2 validation errors for Settings
CORS_ORIGINS
  Field required [type=missing, input_value={'OPENAI_API_KEY': 'sk-7k..._DB_DBNAME': 'postgres'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/missing
CORS_HEADERS
  Field required [type=missing, input_value={'OPENAI_API_KEY': 'sk-7k..._DB_DBNAME': 'postgres'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/missing

In [24]:
import src.config

ValidationError: 2 validation errors for Settings
CORS_ORIGINS
  Field required [type=missing, input_value={'OPENAI_API_KEY': 'sk-7k..._DB_DBNAME': 'postgres'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/missing
CORS_HEADERS
  Field required [type=missing, input_value={'OPENAI_API_KEY': 'sk-7k..._DB_DBNAME': 'postgres'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/missing